Μετατροπή εικόνας σε grayscale και κατωφλίωση


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the image
image = cv2.imread('leaf.jpg')

# Convert to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Thresholding
threshold_value = 220
_, binary_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)

# Plot grayscale image
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(gray_image, cmap='gray')
plt.title('Grayscale Image')
plt.axis('off')

# Plot binary image
plt.subplot(1, 2, 2)
plt.imshow(binary_image, cmap='binary')
plt.title('Binary Image')
plt.axis('off')

plt.show()


Εξαγωγή περιγράμματος του σχήματος με χρήση αλγορίθμου Moore

In [ ]:
def moore_boundary(image):
    height, width = image.shape
    boundary_image = np.zeros((height, width), dtype=int)

    def is_boundary_pixel(x, y):
        if image[x][y] == 0:
            return False
        for i in range(x - 1, x + 2):
            for j in range(y - 1, y + 2):
                if image[i][j] == 0:
                    return True
        return False

    for x in range(1, height - 1):
        for y in range(1, width - 1):
            if is_boundary_pixel(x, y):
                boundary_image[x][y] = 1

    return boundary_image

boundary_image = moore_boundary(binary_image)

plt.imshow(boundary_image, cmap='gray')

Υπολογισμός των περιγραφέων Fourier και ανακατασκευή εικόνας

In [ ]:
# Assuming boundary_image is already loaded correctly as grayscale or BGR
gray = boundary_image if len(boundary_image.shape) == 2 else cv2.cvtColor(boundary_image, cv2.COLOR_BGR2GRAY)

# Calculate the discrete Fourier transform
DFT = cv2.dft(np.float32(gray), flags=cv2.DFT_COMPLEX_OUTPUT)

# Reposition the zero-frequency component to the spectrum's middle
shift = np.fft.fftshift(DFT)
rows, cols = gray.shape

# Function to reconstruct with a specific percentage of Fourier descriptors
def reconstruct_with_percentage(shift, percentage):
    rows, cols = shift.shape[:2]
    center_row, center_col = rows // 2, cols // 2
    
    # Calculate distances from the center
    Y, X = np.ogrid[:rows, :cols]
    distances = np.sqrt((X - center_col)**2 + (Y - center_row)**2)
    
    # Flatten the distances and sort them
    sorted_distances = np.sort(distances.ravel())
    
    # Clamp num_coeffs to the size of sorted_distances
    num_coeffs = min(int((percentage / 100) * (rows * cols)), sorted_distances.size - 1)
    
    # Find the threshold distance for the given percentage
    threshold_distance = sorted_distances[num_coeffs]
    
    # Create a mask based on the threshold distance
    mask = distances <= threshold_distance
    mask = np.repeat(mask[:, :, np.newaxis], 2, axis=2)  # Repeat the mask for both real and imaginary parts
    
    # Apply the mask
    fft_shift_masked = shift * mask
    
    # Inverse shift and inverse DFT
    ifft_shift = np.fft.ifftshift(fft_shift_masked)
    image_reconstructed = cv2.idft(ifft_shift)
    
    # Calculate magnitude
    image_reconstructed = cv2.magnitude(image_reconstructed[:, :, 0], image_reconstructed[:, :, 1])
    
    # Normalize and convert to uint8
    cv2.normalize(image_reconstructed, image_reconstructed, 0, 255, cv2.NORM_MINMAX)
    image_reconstructed = np.uint8(image_reconstructed)
    
    return image_reconstructed

# Percentages to reconstruct with, including 100%
percentages = [100, 50, 10, 1]

# Reconstruct and visualize images for each percentage
plt.figure(figsize=(20, 5))  # Adjust the figure size for better spacing
for i, percentage in enumerate(percentages):
    reconstructed_image = reconstruct_with_percentage(shift, percentage)
    plt.subplot(1, 4, i + 1)  # Adjust to 1 row and 4 columns
    plt.imshow(reconstructed_image, cmap='gray')
    plt.title(f'Reconstructed Image ({percentage}%)')
    plt.xticks([]), plt.yticks([])

plt.tight_layout()
plt.show()


Μετασχηματισμός και ολίσθηση

In [ ]:
# Read the image
image = cv2.imread('leaf.jpg')

# Get the image dimensions
height, width = image.shape[:2]

# 1. Rotate the image by 60 degrees

# Calculate the center of the image
center = (width // 2, height // 2)

# Get the rotation matrix
rotation_matrix = cv2.getRotationMatrix2D(center, 60, 1)  # 60 degrees, scale = 1

# Apply the rotation
rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

# 2. Translate the image by (100px, 60px)

# Create the translation matrix
translation_matrix = np.float32([[1, 0, 100], [0, 1, 60]])

# Apply the translation
translated_image = cv2.warpAffine(rotated_image, translation_matrix, (width, height))

# Convert to grayscale
gray_image = cv2.cvtColor(translated_image, cv2.COLOR_BGR2GRAY)

# Thresholding
threshold_value = 220
_, binary_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)

# Plot the original image
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.axis('off')

# Plot the rotated and translated image
plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(translated_image, cv2.COLOR_BGR2RGB))
plt.title('Rotated & Translated Image')
plt.axis('off')

# Plot the binary image
plt.subplot(1, 3, 3)
plt.imshow(binary_image, cmap='binary')
plt.title('Binary Image')
plt.axis('off')

plt.tight_layout()
plt.show()
